# Perdido en el medio: El problema con los contextos largos

"Independientemente de la arquitectura de tu modelo, existe una degradación sustancial del rendimiento cuando incluyes más de 10 documentos recuperados. En resumen: Cuando los modelos deben acceder a información relevante en medio de contextos largos, tienden a ignorar los documentos proporcionados. Ver: https://arxiv.org/abs/2307.03172

Para evitar este problema, puedes reordenar los documentos después de recuperarlos para evitar la degradación del rendimiento."

---
[Langchain](https://python.langchain.com/docs/modules/data_connection/document_transformers/post_retrieval/long_context_reorder)

## Librerías

In [1]:
from operator import itemgetter

from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_transformers import LongContextReorder
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from src.langchain_docs_loader import LangchainDocsLoader, num_tokens_from_string

load_dotenv()

True

## Carga de datos

In [2]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=10,
    length_function=num_tokens_from_string,
)

documents = LangchainDocsLoader().load()
documents = text_splitter.split_documents(documents)

## Creación de retriever

In [3]:
retriever = Chroma.from_documents(documents, embedding=OpenAIEmbeddings()).as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
        "fetch_k": 50,
    },
)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 1000000 / min. Current: 774651 / min. Contact us through our help center at help.openai.com if you continue to have issues..


## Consulta con el retriever

In [4]:
relevant_docs = retriever.get_relevant_documents(
    "How to use LCEL ainvoke with a retriever?"
)
relevant_docs

[Document(page_content='## LLMRails as a Retriever\u200b\n\nLLMRails, as all the other LangChain vectorstores, is most often used as a LangChain Retriever:\n\n```python\nretriever = llm_rails.as_retriever()\nretriever\n```\n\n```text\n    LLMRailsRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.llm_rails.LLMRails object at 0x107b9c040>, search_type=\'similarity\', search_kwargs={\'k\': 5})\n```\n\n```python\nquery = "What is your approach to national defense"\nretriever.get_relevant_documents(query)[0]\n```', metadata={'description': 'LLMRails is a API platform for building GenAI applications. It provides an easy-to-use API for document indexing and querying that is managed by LLMRails and is optimized for performance and accuracy.', 'language': 'en', 'source': 'https://python.langchain.com/docs/integrations/vectorstores/llm_rails', 'title': 'LLMRails | 🦜️🔗 Langchain'}),
 Document(page_content="QdrantTranslator\\n6. WeaviateTranslator\\n\\nAnd remote retrievers l

## Reordenado de documentos

In [5]:
reordering = LongContextReorder()
reordered_docs = list(reordering.transform_documents(relevant_docs))
reordered_docs

[Document(page_content="QdrantTranslator\\n6. WeaviateTranslator\\n\\nAnd remote retrievers like:\\n\\n1. RemoteLangChainRetriever'}", metadata={'description': "In this tutorial, we are going to use Langchain + Activeloop's Deep Lake with GPT to analyze the code base of the LangChain itself.", 'language': 'en', 'source': 'https://python.langchain.com/docs/use_cases/question_answering/how_to/code/code-analysis-deeplake', 'title': "Use LangChain, GPT and Activeloop's Deep Lake to work with code base | 🦜️🔗 Langchain"}),
 Document(page_content='# Self-querying\n\nA self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to its underlying VectorStore. This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documents but to a

## Uso del reordenador en nuestro pipeline de `Retrival Augmented Generation`

In [6]:
def combine_documents(documents: list[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in documents])


prompt = PromptTemplate.from_template(
    """Given the following text extracts:
-----
{context}
-----
                                      
Answer the following question, if you don't know the answer, just write "I don't know.

Question: {question}"""
)

llm = ChatOpenAI(temperature=0)

stuff_chain = (
    {
        "context": itemgetter("question")
        | retriever
        | reordering.transform_documents
        | combine_documents,
        "question": itemgetter("question"),
    }
    | prompt
    | llm
)

In [7]:
response = stuff_chain.invoke(input={"question": "How to create a chain using LCEL?"}).content
print(response)

To create a chain using LCEL, you can follow these steps:

1. Import the necessary modules and classes from the LangChain library.
2. Define the components of your chain, such as LLMs, prompts, and tools.
3. Use the LCEL syntax to chain together the components in the desired order.
4. Invoke the chain with the input data to get the output.

Here is an example of creating a chain using LCEL:

```python
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.anonymizers import PresidioReversibleAnonymizer

# Define the components
anonymizer = PresidioReversibleAnonymizer()
prompt = PromptTemplate.from_template(template="{anonymized_text}")
llm = ChatOpenAI(temperature=0)

# Chain the components together
chain = {"anonymized_text": anonymizer.anonymize} | prompt | llm

# Invoke the chain with input data
text = "This is a sample text."
response = chain.invoke(text)

# Get the output
output = response.content
print(output)
```

In this